# **Drive mount**

In [6]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
PROJECT_PATH = ROOT+'/My Drive/Fall2020/My_Capstone/'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd "{PROJECT_PATH}"
#!git clone -b Pritam "{GIT_PATH}"
%cd "Edge"


/content/drive/.shortcut-targets-by-id/1InqMx1dIPSCnt1piwJDHa41grcQUFVAI/My_Capstone
/content/drive/.shortcut-targets-by-id/1InqMx1dIPSCnt1piwJDHa41grcQUFVAI/My_Capstone/Edge


In [8]:

def quantization_eval_results(model_name,train_set,test_set,batch_size,criterion):
  results = quantization_corrected(model_name)
  train_loss_list = []
  train_accuracy_list = []
  test_loss_list = []
  test_accuracy_list = []
  for i in results["model_artifact"]:
    train_loss, train_accuracy = evaluate(model=i, 
                                        test_set = train_set,
                                        batch_size=batch_size, 
                                        criterion=criterion,
                                        ep=0) 
    test_loss, test_accuracy = evaluate(model=i, 
                                        test_set = test_set,
                                        batch_size=batch_size, 
                                        criterion=criterion,
                                        ep=0)  
    train_loss_list.append(train_loss.item())
    test_loss_list.append(test_loss.item())
    train_accuracy_list.append(train_accuracy)
    test_accuracy_list.append(test_accuracy)
    # print("End of training, test loss =  {}, test accuracy = {} \n".format(train_loss, train_accuracy))
    # print("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
  results["train_loss"] = train_loss_list
  results["train_acc"] = train_accuracy_list
  results["test_loss"] = test_loss_list
  results["test_acc"] = test_accuracy_list
  return results



def quantization_corrected(model_name, method='all'):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    precision = [16, 12, 8, 4, 2, 1, 0]
    model_object = 'model_artifacts/' + model_name
    results = pd.DataFrame(columns=['model', 'method', 'precision', 'model_artifact',
                                    'train_loss', 'train_acc', 'test_loss', 'test_acc'])
    model = torch.load(model_object, map_location=torch.device(device))
    results = results.append(
        {'model': model_name, 'method': 'no rounding', 'precision': 32, 'model_artifact': model},
        ignore_index=True
    )

    # normal rounding
    if method == 'rounding' or method == 'all':
        for p in precision:
            weights = dict()
            model = torch.load(model_object, map_location=torch.device(device))

            for name, params in model.named_parameters():
                weights[name] = params.clone()
            for w in weights:
                weights[w] = torch.round(weights[w] * 10**p) / (10**p)
            for name, params in model.named_parameters():
                params.data.copy_(weights[name])

            results = results.append(
                {'model': model_name, 'method': 'rounding', 'precision': p, 'model_artifact': model},
                ignore_index=True
            )

    # mid-rise quantization
    if method == 'mid-rise' or method == 'all':
        for p in precision:
            weights = dict()
            model = torch.load(model_object, map_location=torch.device(device))

            for name, params in model.named_parameters():
                weights[name] = params.clone()
            for w in weights:
                if len(weights[w]) == 1:
                    delta = weights[w] / 2**p
                else:
                    delta = (torch.max(weights[w]) - torch.min(weights[w])) / 2**p
                weights[w] = delta * torch.floor(weights[w]/delta) + 0.5
            for name, params in model.named_parameters():
                params.data.copy_(weights[name])

            results = results.append(
                {'model': model_name, 'method': 'mid-rise', 'precision': p, 'model_artifact': model},
                ignore_index=True
            )

    # stochastic rounding
    if method == 'stochastic' or method == 'all':
        for p in precision:
            weights = dict()
            model = torch.load(model_object, map_location=torch.device(device))

            for name, params in model.named_parameters():
                weights[name] = params.clone()
            for w in weights:
                fix = torch.sign(weights[w]) * 10**p
                weights[w] = weights[w] * fix
                diff = weights[w] - torch.floor(weights[w])
                round = torch.floor(weights[w]) + torch.tensor(np.random.binomial(1, diff.cpu().data.numpy()),device=device)
                weights[w] = round / fix
            for name, params in model.named_parameters():
                params.data.copy_(weights[name])

            results = results.append(
                {'model': model_name, 'method': 'stochastic', 'precision': p, 'model_artifact': model},
                ignore_index=True
            )

    return results

# **Model = ConvNet, Data = FashionMNIST data**
Val Accuracy=92%

In [9]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_fmnist_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
  ])

  transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=(0.5), std=(0.5))
  ])

  if train == True:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/fmnist_cnn_train.log", "a")
  val_log = open("log/fmnist_cnn_val.log", "a")
  test_log = open("log/fmnist_cnn_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
### config params
  output_classes = 10
  learning_rate = 0.0008
  batch_size = 16
  epochs = 5
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'fmnist_cnn_model.pt'
  criterion = nn.CrossEntropyLoss()
####

  train_set, val_set, test_set = None, None, None
  train_set = get_fmnist_dataset(train = True)
  val_set = get_fmnist_dataset(train = False)
  print("Image Shape: ",next(iter(train_set))[0].shape)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN(output_classes)
    #model = Network()
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = False)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion)
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3] +".csv")


if __name__ == "__main__":
  main(load_model = True)


Image Shape:  torch.Size([3, 32, 32])



Running evaluation on loaded model, validation loss = 0.220295, validation accuracy = 0.920500



                  model       method  ...     test_loss test_acc
0   fmnist_cnn_model.pt  no rounding  ...  2.202947e-01   0.9205
1   fmnist_cnn_model.pt     rounding  ...  2.202944e-01   0.9205
2   fmnist_cnn_model.pt     rounding  ...  2.202946e-01   0.9205
3   fmnist_cnn_model.pt     rounding  ...  2.202945e-01   0.9205
4   fmnist_cnn_model.pt     rounding  ...  2.203388e-01   0.9205
5   fmnist_cnn_model.pt     rounding  ...  2.181943e-01   0.9214
6   fmnist_cnn_model.pt     rounding  ...  2.683032e-01   0.9057
7   fmnist_cnn_model.pt     rounding  ...  2.302602e+00   0.1000
8   fmnist_cnn_model.pt     mid-rise  ...  5.345287e+14   0.1000
9   fmnist_cnn_model.pt     mid-rise  ...  5.332053e+14   0.1000
10  fmnist_cnn_model.pt     mid-rise  ...  5.153026e+14   0.1000
11  fmnist_cnn_model.pt     mid-rise  ...  2.738749e+14   0.1000
12  fmnist_cnn_model.pt     mid-rise  ...  1.061383e+14   0.1000
13  fmnist_cnn_model.pt     mid-rise  ...  3.434334e+12   0.1000
14  fmnist_cnn_model.pt 

# **Model = Resnet9, Data = FashionMNIST data**
Val Accuracy=93%



In [10]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_resnet9
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_fmnist_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
  ])

  transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=(0.5), std=(0.5))
  ])

  if train == True:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset

def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-5

def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/fmnist_resnet9_train.log", "a")
  val_log = open("log/fmnist_resnet9_val.log", "a")
  test_log = open("log/fmnist_resnet9_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)

  # trainloader
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=max_lr, weight_decay=weight_decay)
  sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      nn.utils.clip_grad_value_(model.parameters(), grad_clip)
      optimizer.step()
      sched.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
### config params
  output_classes = 10
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'fmnist_resnet9_model.pt'
  criterion = nn.CrossEntropyLoss()
####

  train_set, val_set, test_set = None, None, None
  train_set = get_fmnist_dataset(train = True)
  val_set = get_fmnist_dataset(train = False)
  print("Image Shape: ",next(iter(train_set))[0].shape)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN_resnet9(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion)
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3] +".csv")


if __name__ == "__main__":
  main(load_model = False)


Image Shape:  torch.Size([3, 32, 32])
Device:  cuda:0
Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,856
       BatchNorm2d-5          [-1, 128, 32, 32]             256
              ReLU-6          [-1, 128, 32, 32]               0
         MaxPool2d-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1,


Step = 3750, validation loss =  0.442, validation accuracy = 0.868



Step = 7500, validation loss =  0.384, validation accuracy = 0.865



Step = 11250, validation loss =  0.312, validation accuracy = 0.894



Step = 15000, validation loss =  0.332, validation accuracy = 0.885


...


Step = 26250, validation loss =  0.257, validation accuracy = 0.913



Step = 30000, validation loss =  0.225, validation accuracy = 0.917



Step = 33750, validation loss =  0.215, validation accuracy = 0.924



Step = 37500, validation loss =  0.212, validation accuracy = 0.925




                      model       method  ...     test_loss test_acc
0   fmnist_resnet9_model.pt  no rounding  ...  2.124799e-01   0.9248
1   fmnist_resnet9_model.pt     rounding  ...  2.124799e-01   0.9248
2   fmnist_resnet9_model.pt     rounding  ...  2.124798e-01   0.9248
3   fmnist_resnet9_model.pt     rounding  ...  2.124798e-01   0.9248
4   fmnist_resnet9_model.pt     rounding  ...  2.124830e-01   0.9248
5   fmnist_resnet9_model.pt     rounding  ...  2.124789e-01   0.9247
6   fmnist_resnet9_model.pt     rounding  ...  2.195453e-01   0.9223
7   fmnist_resnet9_model.pt     rounding  ...  9.040912e+00   0.2275
8   fmnist_resnet9_model.pt     mid-rise  ...           NaN   0.1084
9   fmnist_resnet9_model.pt     mid-rise  ...           NaN   0.1086
10  fmnist_resnet9_model.pt     mid-rise  ...           NaN   0.1006
11  fmnist_resnet9_model.pt     mid-rise  ...           inf   0.1000
12  fmnist_resnet9_model.pt     mid-rise  ...  5.011635e+31   0.1000
13  fmnist_resnet9_model.pt     m

# **Model = Resnet50, Data = FashionMNIST data**
Val Accuracy=78%

In [11]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_resnet50
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_fmnist_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
  ])

  transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=(0.5), std=(0.5))
  ])

  if train == True:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/fmnist_resnet50_train.log", "a")
  val_log = open("log/fmnist_resnet50_val.log", "a")
  test_log = open("log/fmnist_resnet50_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
### config params
  output_classes = 10
  learning_rate = 0.0009
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'fmnist_resnet50_model.pt'
  criterion = nn.CrossEntropyLoss()
####

  train_set, val_set, test_set = None, None, None
  train_set = get_fmnist_dataset(train = True)
  val_set = get_fmnist_dataset(train = False)
  print("Image Shape: ",next(iter(train_set))[0].shape)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN_resnet50(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion)
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3] +".csv")


if __name__ == "__main__":
  main(load_model = False)


Image Shape:  torch.Size([3, 32, 32])


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



Device:  cuda:0
Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      B


Step = 3750, validation loss =  0.799, validation accuracy = 0.741



Step = 7500, validation loss =  0.809, validation accuracy = 0.737


...


Step = 26250, validation loss =  0.681, validation accuracy = 0.770



Step = 30000, validation loss =  0.727, validation accuracy = 0.774



Step = 33750, validation loss =  0.644, validation accuracy = 0.779



Step = 37500, validation loss =  0.631, validation accuracy = 0.785




                       model       method  ...     test_loss test_acc
0   fmnist_resnet50_model.pt  no rounding  ...  6.306421e-01   0.7851
1   fmnist_resnet50_model.pt     rounding  ...  6.306417e-01   0.7851
2   fmnist_resnet50_model.pt     rounding  ...  6.306421e-01   0.7851
3   fmnist_resnet50_model.pt     rounding  ...  6.306412e-01   0.7851
4   fmnist_resnet50_model.pt     rounding  ...  6.309021e-01   0.7855
5   fmnist_resnet50_model.pt     rounding  ...  8.223672e-01   0.7041
6   fmnist_resnet50_model.pt     rounding  ...  1.533778e+01   0.1000
7   fmnist_resnet50_model.pt     rounding  ...  2.302602e+00   0.1000
8   fmnist_resnet50_model.pt     mid-rise  ...           NaN   0.1000
9   fmnist_resnet50_model.pt     mid-rise  ...           NaN   0.1000
10  fmnist_resnet50_model.pt     mid-rise  ...           NaN   0.1000
11  fmnist_resnet50_model.pt     mid-rise  ...           NaN   0.1000
12  fmnist_resnet50_model.pt     mid-rise  ...           NaN   0.1000
13  fmnist_resnet50

# **Model = VGG16, Data= FashionMNIST**
Val Accuracy=65%

In [12]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_vgg16
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_fmnist_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
  ])

  transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=(0.5), std=(0.5))
  ])

  if train == True:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.FashionMNIST(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/fmnist_vgg16_train.log", "a")
  val_log = open("log/fmnist_vgg16_val.log", "a")
  test_log = open("log/fmnist_vgg16_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-6, momentum=0.9, nesterov=True)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
### config params
  output_classes = 10
  learning_rate = 0.01
  batch_size = 32
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'fmnist_vgg16_model.pt'
  criterion = nn.CrossEntropyLoss()
####

  train_set, val_set, test_set = None, None, None
  train_set = get_fmnist_dataset(train = True)
  val_set = get_fmnist_dataset(train = False)
  print("Image Shape: ",next(iter(train_set))[0].shape)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN_vgg16(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion)
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3] +".csv")


if __name__ == "__main__":
  main(load_model = False)


Image Shape:  torch.Size([3, 32, 32])


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



Device:  cuda:0
Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
       


Step = 1875, validation loss =  1.715, validation accuracy = 0.745



Step = 3750, validation loss =  1.748, validation accuracy = 0.712


...


Step = 13125, validation loss =  1.860, validation accuracy = 0.601



Step = 15000, validation loss =  1.859, validation accuracy = 0.602



Step = 16875, validation loss =  1.788, validation accuracy = 0.673



Step = 18750, validation loss =  1.757, validation accuracy = 0.704



RuntimeError: ignored